## Análisis del modelo matematico propuesto

In [58]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#  Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

In [59]:
from datetime import datetime, timedelta
import random

# Establece la fecha de inicio y fin del rango
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d')

# Calcula la diferencia de días entre las fechas
delta = end_date - start_date

# Genera un número aleatorio de días para añadir a la fecha de inicio
random_days = random.randint(0, delta.days)

# Calcula la fecha aleatoria
random_date = start_date + timedelta(days=random_days)

# Formatea la fecha al formato deseado
Fecha = random_date.strftime('%Y-%m-%d')
print(Fecha)


2023-05-01


In [89]:
#  Cargar los datos del archivo CSV
Fecha = '2023-01-02'
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
row_day_solcast = df_solcast.loc[df_solcast['Date'] == Fecha]
row_day_fronius = df_SFV.loc[df_SFV['Date'] == Fecha]
# reiniciar el index
row_day_solcast = row_day_solcast.reset_index(drop=True)
row_day_fronius = row_day_fronius.reset_index(drop=True)
# Odenernar row_day_solcast de menor a mayor hora
row_day_solcast = row_day_solcast.sort_values(by='Time')
row_day_solcast = row_day_solcast.reset_index(drop=True)
# Crear un nuevo dataframe con los datos de solcast y fronius a partir del indice 
df = pd.DataFrame()
df['Time'] = row_day_solcast['Time']
df['GHI'] = row_day_solcast['GHI']
df['air_temp'] = row_day_solcast['air_temp']
df['cloud_opacity'] = row_day_solcast['cloud_opacity']
df['Producción_fotovoltaica_SFV'] = row_day_fronius['Producción_fotovoltaica_SFV']
df['clearsky_ghi'] = row_day_solcast['clearsky_ghi']
df['clearsky_gti'] = row_day_solcast['clearsky_gti']

In [90]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

#  Cargar los datos del archivo CSV
import ipywidgets as widgets
from IPython.display import display

def plot_signal(alpha, beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Energía calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity']) * (1 - alpha*df['air_temp'])

    # Energía calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)
    
    plt.figure(figsize=(15, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray', alpha=0.2)
    plt.title(f'Producción fotovoltaica calculada de {Fecha} con: Alpha={alpha}, Beta={beta}  \nE = Irr * efficiency *  area * catidad_pv * (1-beta * N) * (1-alpha*T) * Delta_t')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.xticks([100, 150, 200])
    # plt.grid(True)
    plt.show()
    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    # Horas de brillo solar
    hbs = energia_fronius / (30*0.54)
    print("Horas de brillo solar =", hbs/1000)
    print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia = df['P_cloud_opacity_temp']*(5/60)
    print("Energía calculada solacast [kWh/día] =",energia.sum()/1000)
    # error porcentual
    error = abs(  energia.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    print('Error porcentual:', error, '%')

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal, alpha=amplitude_slider, beta=frequency_slider)

interactive(children=(FloatSlider(value=0.1, description='alpha:', max=1.0), FloatSlider(value=0.3, descriptio…

<function __main__.plot_signal(alpha, beta)>

In [91]:
# normalizar producción fotovoltaica
print(df['Producción_fotovoltaica_SFV'].max())

df['Producción_fotovoltaica_SFV'] = df['Producción_fotovoltaica_SFV'] / df['Producción_fotovoltaica_SFV'].max()


energia_SFV = df['Producción_fotovoltaica_SFV']
print("Energía calculada SFV [kWh/día] =",energia_SFV.sum()/1000)
print("Energía calculada SFV [kWh/día] =",energia_SFV.sum()*1271.64/1000)

461.0
Energía calculada SFV [kWh/día] = 0.060550130151843816
Energía calculada SFV [kWh/día] = 76.99796750629068


## Análisis con clearsky_ghi

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

#  Cargar los datos del archivo CSV
import ipywidgets as widgets
from IPython.display import display

def plot_signal(alpha, beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Potencia calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity']) * (1 - alpha*df['air_temp'])

    # Potencia calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)

    # Potencia clearsky_ghi
    df['P_clearsky_ghi'] = df['clearsky_ghi'] *  area * catidad_pv * efficiency

    # Valor máximo de df['P_clearsky_ghi']
    maximo = df['P_clearsky_ghi'].max()
    # valor medio de df['P_clearsky_ghi']
    medio = df['P_clearsky_ghi'].mean()
    # medio = maximo / 2





    
    plt.figure(figsize=(17, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # agregar clearsky_ghi
    plt.plot(df['Time'], df['P_clearsky_ghi'] , label='Clearsky')
    # grafica una linea en la hora del  máximo de df['P_clearsky_ghi']
    plt.axvline(x=df['Time'][df['P_clearsky_ghi'].idxmax()], color='r', linestyle='--', label='Máximo Clearsky')
    # grafica una linea en el valor medio de df['P_clearsky_ghi']
    plt.axhline(y=medio, color='g', linestyle='--', label='Medio Clearsky')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray', alpha=0.2)
   

    plt.title(f'Producción fotovoltaica calculada de {Fecha} con: Alpha={alpha}, Beta={beta}  \nE = Irr * efficiency *  area * catidad_pv * (1-beta * N) * (1-alpha*T) * Delta_t')
    plt.xlabel('Tiempo')
    plt.ylabel('Potencia [W]')
    plt.legend()
    plt.xticks([100, 150, 200])
    plt.grid(True)
    plt.show()




    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia_solcast = df['P_cloud_opacity_temp']*(5/60)
    print("Producción fotovoltaica solcast [kWh/día] =",energia_solcast.sum()/1000)
    # error porcentual
    error = abs(  energia_solcast.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    print('Error porcentual solcast vs SFV :', error, '%')

    # energía calculada a partir de clearsky
    energia_clearsky = df['P_clearsky_ghi']*(5/60)
    # print("Producción fotovoltaica clearsky [kWh/día] =",energia_clearsky.sum()/1000)
    # error porcentual clearsky vs energia_solcast
    # imprimir un espacio 
    print()
    error = abs(  energia_clearsky.sum() - energia_solcast.sum()) / energia_solcast.sum() * 100
    print('Error porcentual clearsky vs solacast :', error, '%')
    
 

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal, alpha=amplitude_slider, beta=frequency_slider)

interactive(children=(FloatSlider(value=0.0, description='alpha:', max=1.0), FloatSlider(value=0.0, descriptio…

<function __main__.plot_signal(alpha, beta)>

## Análisis con grafica de  nubosidad y temperatura 

In [46]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

#  Cargar los datos del archivo CSV
import ipywidgets as widgets
from IPython.display import display

def plot_signal(alpha, beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Potencia calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity']) * (1 - alpha*df['air_temp'])

    # Potencia calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)

    # Potencia clearsky_ghi
    df['P_clearsky_ghi'] = df['clearsky_ghi'] *  area * catidad_pv * efficiency

    # Valor máximo de df['P_clearsky_ghi']
    maximo = df['P_clearsky_ghi'].max()
    # valor medio de df['P_clearsky_ghi']
    medio = df['P_clearsky_ghi'].mean()
    # medio = maximo / 2





    
    plt.figure(figsize=(17, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # agregar clearsky_ghi
    plt.plot(df['Time'], df['P_clearsky_ghi'] , label='Clearsky')
    # grafica una linea en la hora del  máximo de df['P_clearsky_ghi']
    plt.axvline(x=df['Time'][df['P_clearsky_ghi'].idxmax()], color='r', linestyle='--', label='Máximo Clearsky')
    # grafica una linea en el valor medio de df['P_clearsky_ghi']
    plt.axhline(y=medio, color='g', linestyle='--', label='Medio Clearsky')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray', alpha=0.2)

    # # grafico de cloud_opacity y air_temp en el tiempo
    # plt.plot(df['Time'], df['cloud_opacity']*df['P_cloud_opacity_temp'].max(), label='cloud_opacity')
    # plt.plot(df['Time'], df['air_temp']*df['P_cloud_opacity_temp'].max(), label='air_temp')
   

    plt.title(f'Producción fotovoltaica calculada de {Fecha} con: Alpha={alpha}, Beta={beta}  \nE = Irr * efficiency *  area * catidad_pv * (1-beta * N) * (1-alpha*T) * Delta_t')
    plt.xlabel('Tiempo')
    plt.ylabel('Potencia [W]')
    plt.legend()
    plt.xticks([100, 150, 200])
    plt.grid(True)
    plt.show()




    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia_solcast = df['P_cloud_opacity_temp']*(5/60)
    print("Producción fotovoltaica solacast [kWh/día] =",energia_solcast.sum()/1000)
    # error porcentual
    error = abs(  energia_solcast.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    print('Error porcentual solacast vs SFV :', error, '%')

    # energía calculada a partir de clea rsky
    energia_clearsky = df['P_clearsky_ghi']*(5/60)
    # print("Producción fotovoltaica clearsky [kWh/día] =",energia_clearsky.sum()/1000)
    # error porcentual clearsky vs energia_solcast
    # imprimir un espacio 
    print()
    error = abs(  energia_clearsky.sum() - energia_solcast.sum()) / energia_solcast.sum() * 100
    print('Error porcentual clearsky vs solacast :', error, '%')
    
 

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal, alpha=amplitude_slider, beta=frequency_slider)

interactive(children=(FloatSlider(value=0.0, description='alpha:', max=1.0), FloatSlider(value=0.0, descriptio…

<function __main__.plot_signal(alpha, beta)>

## Análisis solo con el componente de nubosidad 


In [47]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

#  Cargar los datos del archivo CSV
import ipywidgets as widgets
from IPython.display import display

def plot_signal(beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Potencia calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity'])* (1-0.1*df['air_temp'])

    # Potencia calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)

    # Potencia clearsky_ghi
    df['P_clearsky_ghi'] = df['clearsky_ghi'] *  area * catidad_pv * efficiency

    # Valor máximo de df['P_clearsky_ghi']
    maximo = df['P_clearsky_ghi'].max()
    # valor medio de df['P_clearsky_ghi']
    medio = df['P_clearsky_ghi'].mean()
    # medio = maximo / 2





    
    plt.figure(figsize=(17, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # agregar clearsky_ghi
    plt.plot(df['Time'], df['P_clearsky_ghi'] , label='Clearsky')
    # grafica una linea en la hora del  máximo de df['P_clearsky_ghi']
    plt.axvline(x=df['Time'][df['P_clearsky_ghi'].idxmax()], color='r', linestyle='--', label='Máximo Clearsky')
    # grafica una linea en el valor medio de df['P_clearsky_ghi']
    plt.axhline(y=medio, color='g', linestyle='--', label='Medio Clearsky')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray')

    # # grafico de cloud_opacity y air_temp en el tiempo
    # plt.plot(df['Time'], df['cloud_opacity']*df['P_cloud_opacity_temp'].max(), label='cloud_opacity')
    # plt.plot(df['Time'], df['air_temp']*df['P_cloud_opacity_temp'].max(), label='air_temp')
   

    plt.title(f'Producción fotovoltaica calculada de {Fecha} con: Beta={beta}  \nE = Irr * efficiency *  area * catidad_pv * (1-beta * N) (1-0.1*T)* Delta_t')
    plt.xlabel('Tiempo')
    plt.ylabel('Potencia [W]')
    plt.legend()
    plt.xticks([100, 150, 200])
    plt.grid(True)
    plt.show()



    #Date
    print("Fecha:","'",Fecha,"'")
    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    # print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    print("E_SVF =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia_solcast = df['P_cloud_opacity_temp']*(5/60)
    # print("Producción fotovoltaica solacast [kWh/día] =",energia_solcast.sum()/1000)
    print("E_solacast=",energia_solcast.sum()/1000)
    # alpah & beta
    print("alpha = 0.1")
    print("beta =", beta)

    # error porcentual
    error = abs(  energia_solcast.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    # print('Error porcentual solacast vs SFV =', error, '%')
    print('error_solacast_SFV =', error)

    # energía calculada a partir de clea rsky
    energia_clearsky = df['P_clearsky_ghi']*(5/60)
    # print("Producción fotovoltaica clearsky [kWh/día] =",energia_clearsky.sum()/1000)
    # error porcentual clearsky vs energia_solcast
    # imprimir un espacio 
    # print()
    error = abs(  energia_clearsky.sum() - energia_solcast.sum()) / energia_solcast.sum() * 100
    print('error_clearsky_solacast =', error)
    
 

# Definir sliders
# amplitude_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal,  beta=frequency_slider)

interactive(children=(FloatSlider(value=0.0, description='beta:', max=1.0), Output()), _dom_classes=('widget-i…

<function __main__.plot_signal(beta)>

In [48]:
import csv
import os

# Pregunta al usuario si desea continuar
respuesta = input("¿Deseas guardar los datos en el archivo CSV? (y/n): ")

# Si la respuesta es afirmativa, procede con el guardado de datos
if respuesta.lower() == 'y' or respuesta.lower() == 'n':

    # Define los valores de tus variables
    Fecha: ' 2023-02-19 '
    E_SVF = 57.64743000000001
    E_solacast= 56.53813967029868
    alpha = 0.1
    beta = 0.6
    error_solacast_SFV = 1.9242667534377946
    error_clearsky_solacast = 123.57173719303279

    # Nombre del archivo CSV donde guardarás los datos
    filename = 'datos_variables.csv'

    # Verifica si el archivo ya existe
    file_exists = os.path.isfile(filename)

    # Abre el archivo en modo de añadir ('a') si existe, o en modo de escritura ('w') si no
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        # Si el archivo no existe, escribe la cabecera
        if not file_exists:
            writer.writerow(['Fecha','E_SVF', 'E_solacast', 'alpha', 'beta', 'error_solacast_SFV', 'error_clearsky_solacast'])

        # Escribe los datos de las variables en el archivo
        writer.writerow([Fecha, E_SVF, E_solacast, alpha, beta, error_solacast_SFV, error_clearsky_solacast])

    print("Datos guardados correctamente.")
else:
    print("Operación cancelada.")


Operación cancelada.


## Optimización

In [49]:
from scipy.optimize import minimize

def error_porcentual(alpha_beta):
    alpha, beta = alpha_beta
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    P_cloud_opacity_temp = df['GHI'] * efficiency * area * catidad_pv * (1 - beta * df['cloud_opacity']) * (1 - alpha * df['air_temp'])
    energia = P_cloud_opacity_temp * (5/60)
    error = abs(energia.sum() - df['Producción_fotovoltaica_SFV'].sum()) / df['Producción_fotovoltaica_SFV'].sum() * 100
    return error

# Suponiendo que tienes valores iniciales para alpha y beta
alpha_init = 0.1
beta_init = 0.1
initial_guess = [alpha_init, beta_init]

resultado = minimize(error_porcentual, initial_guess, bounds=((0, 1), (0, 1)))
alpha_optimo, beta_optimo = resultado.x

print("Alpha óptimo:", alpha_optimo)
print("Beta óptimo:", beta_optimo)
print("Error porcentual mínimo:", resultado.fun, "%")


Alpha óptimo: 0.3476231537065768
Beta óptimo: 0.34762315373162234
Error porcentual mínimo: 3.7008024285965805e-07 %


In [50]:
from scipy.optimize import minimize
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df con las columnas 'GHI', 'cloud_opacity', 'air_temp' y 'Producción_fotovoltaica_SFV'

def error_porcentual(alpha_beta):
    alpha, beta = alpha_beta
    efficiency = 0.2094  # Eficiencia del panel solar
    cantidad_pv = 30
    area = 2.274 * 1.134
    P_cloud_opacity_temp = df['GHI'] * efficiency * area * cantidad_pv * (1 - beta * df['cloud_opacity']) * (1 - alpha * df['air_temp'])
    energia = P_cloud_opacity_temp * (5 / 60)
    error = abs(energia.sum() - df['Producción_fotovoltaica_SFV'].sum()) / df['Producción_fotovoltaica_SFV'].sum() * 100
    return error

# Suponiendo que tienes valores iniciales para alpha y beta
alpha_init = 0.1
beta_init = 0.1
initial_guess = [alpha_init, beta_init]

# Definiendo las restricciones para alpha y beta
bounds = [(0, 1), (0, 1)]

resultado = minimize(error_porcentual, initial_guess, bounds=bounds)
alpha_optimo, beta_optimo = resultado.x

print("Alpha óptimo:", alpha_optimo)
print("Beta óptimo:", beta_optimo)
print("Error porcentual mínimo:", resultado.fun, "%")


Alpha óptimo: 0.3476231537065768
Beta óptimo: 0.34762315373162234
Error porcentual mínimo: 3.7008024285965805e-07 %
